In [81]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [82]:
train_data_path = "/home/akhilesh/DataScience/FlightPricePrediction/Data_Train.xlsx"
test_data_path = "/home/akhilesh/DataScience/FlightPricePrediction/Test_set.xlsx"

train_data = pd.read_excel(train_data_path)


In [83]:
train_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [84]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [85]:
train_data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

There is one null value in Route and Total Stop. We can safely remove it without it affecting our models too much.

In [86]:
train_data.dropna(inplace=True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10682 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10682 non-null  object
 1   Date_of_Journey  10682 non-null  object
 2   Source           10682 non-null  object
 3   Destination      10682 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10682 non-null  object
 6   Arrival_Time     10682 non-null  object
 7   Duration         10682 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10682 non-null  object
 10  Price            10682 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 1001.4+ KB


Next, we can convert the appropriate columns to datetime datatype and derive more useful info like hour, month, etc. Arrival_Time has to be formated as well.

In [87]:
train_data[['Date_of_Journey','Dep_Time','Arrival_Time']].head()
train_data[['Date_of_Journey','Dep_Time','Arrival_Time']].info()


,Date_of_Journey,Dep_Time,Arrival_Time
0,24/03/2019,22:20,01:10 22 Mar
1,1/05/2019,05:50,13:15
2,9/06/2019,09:25,04:25 10 Jun
3,12/05/2019,18:05,23:30
4,01/03/2019,16:50,21:35


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10682 entries, 0 to 10682
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date_of_Journey  10682 non-null  object
 1   Dep_Time         10682 non-null  object
 2   Arrival_Time     10682 non-null  object
dtypes: object(3)
memory usage: 333.8+ KB


In [88]:
data=train_data.copy()

In [89]:
for f in ['Date_of_Journey','Dep_Time','Arrival_Time']:
    data[f]= pd.to_datetime(data[f])

data[['Date_of_Journey','Dep_Time','Arrival_Time']].head()
data[['Date_of_Journey','Dep_Time','Arrival_Time']].tail()

,Date_of_Journey,Dep_Time,Arrival_Time
0,2019-03-24,2023-08-18 22:20:00,2023-03-22 01:10:00
1,2019-01-05,2023-08-18 05:50:00,2023-08-18 13:15:00
2,2019-09-06,2023-08-18 09:25:00,2023-06-10 04:25:00
3,2019-12-05,2023-08-18 18:05:00,2023-08-18 23:30:00
4,2019-01-03,2023-08-18 16:50:00,2023-08-18 21:35:00


,Date_of_Journey,Dep_Time,Arrival_Time
10678,2019-09-04,2023-08-18 19:55:00,2023-08-18 22:25:00
10679,2019-04-27,2023-08-18 20:45:00,2023-08-18 23:20:00
10680,2019-04-27,2023-08-18 08:20:00,2023-08-18 11:20:00
10681,2019-01-03,2023-08-18 11:30:00,2023-08-18 14:10:00
10682,2019-09-05,2023-08-18 10:55:00,2023-08-18 19:15:00


Whops! Current date has been filled into the missing values in Departure and Arrival Times. Pandas does not allow modification of year only or day only. However, we can extract the hour and min and then drop the original ones.


### Extracting Arrtibutes

In [91]:
data["Arrival_Hour"]=data["Arrival_Time"].dt.hour
data["Arrival_Day"]=data["Arrival_Time"].dt.day
data["Arrival_Min"]=data["Arrival_Time"].dt.minute

data["Dep_Hour"]=data["Dep_Time"].dt.hour
data["Dep_Day"]=data["Dep_Time"].dt.day
data["Dep_Min"]=data["Dep_Time"].dt.minute

data["Journey_Day"]=data["Date_of_Journey"].dt.day
data["Journey_Month"]=data["Date_of_Journey"].dt.month
data["Journey_Year"]=data["Date_of_Journey"].dt.year
data["Journey_DOW"]=data["Date_of_Journey"].dt.dayofweek 

data.drop(['Arrival_Time','Dep_Time'], axis=1,inplace=True)

In [92]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10682 entries, 0 to 10682
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Airline          10682 non-null  object        
 1   Date_of_Journey  10682 non-null  datetime64[ns]
 2   Source           10682 non-null  object        
 3   Destination      10682 non-null  object        
 4   Route            10682 non-null  object        
 5   Duration         10682 non-null  object        
 6   Total_Stops      10682 non-null  object        
 7   Additional_Info  10682 non-null  object        
 8   Price            10682 non-null  int64         
 9   Arrival_Hour     10682 non-null  int64         
 10  Arrival_Day      10682 non-null  int64         
 11  Arrival_Min      10682 non-null  int64         
 12  Dep_Hour         10682 non-null  int64         
 13  Dep_Day          10682 non-null  int64         
 14  Dep_Min          10682 non-null  int64

,Airline,Date_of_Journey,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Arrival_Hour,Arrival_Day,Arrival_Min,Dep_Hour,Dep_Day,Dep_Min,Journey_Day,Journey_Month,Journey_Year,Journey_DOW
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897,1,22,10,22,18,20,24,3,2019,6
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662,13,18,15,5,18,50,5,1,2019,5
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882,4,10,25,9,18,25,6,9,2019,4
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218,23,18,30,18,18,5,5,12,2019,3
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302,21,18,35,16,18,50,3,1,2019,3


We can also convert Duration column to Minutes. 

In [ ]:
def duration_to_min(string):

    if ('m' not in string):
        total = int(string[:-1])*60

    
    elif ('h' not in string):
        total = string[:-1]
        
    else:
        hour = int(string.split('h')[0])
        min1 = string.split(' ')[1]
        min2 = int(min1[:-1])
        total = hour*60 + min2
        

    return total

In [93]:
data['Duration'] = data['Duration'].apply(duration_to_min).astype(int)


In [94]:
data['Duration'].head()

0     170
1     445
2    1140
3     325
4     285
Name: Duration, dtype: int64

In [95]:
data['Additional_Info'].describe()
data['Total_Stops'].unique()


count       10682
unique         10
top       No info
freq         8344
Name: Additional_Info, dtype: object

array(['non-stop', '2 stops', '1 stop', '3 stops', '4 stops'],
      dtype=object)

Additional Info has 'No Info' 8344 times out of 10682. We can drop it. 


Total Stops has 5 unique values. They can be replaced with 0, 1, 2, 3 ,4 ,5. This is also an example of **Label Encoding**.

In [97]:
data.drop('Additional_Info',axis=1,inplace=True)

stop = {'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, '4 stops':4}
data['Total_Stops'] = data['Total_Stops'].map(stop).astype(int)

KeyError: "['Additional_Info'] not found in axis"

In [98]:
data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Duration,Total_Stops,Price,Arrival_Hour,Arrival_Day,Arrival_Min,Dep_Hour,Dep_Day,Dep_Min,Journey_Day,Journey_Month,Journey_Year,Journey_DOW
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,170,0,3897,1,22,10,22,18,20,24,3,2019,6
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,445,2,7662,13,18,15,5,18,50,5,1,2019,5
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,1140,2,13882,4,10,25,9,18,25,6,9,2019,4
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,325,1,6218,23,18,30,18,18,5,5,12,2019,3
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,285,1,13302,21,18,35,16,18,50,3,1,2019,3


In [99]:
data.infer_objects().dtypes 
#Infering the datatypes for future use. Ideally this is done right after importing data.
#Somehow it did not work when I tried then. As long as it is done before exploring data, it is ok.

Airline                    object
Date_of_Journey    datetime64[ns]
Source                     object
Destination                object
Route                      object
Duration                    int64
Total_Stops                 int64
Price                       int64
Arrival_Hour                int64
Arrival_Day                 int64
Arrival_Min                 int64
Dep_Hour                    int64
Dep_Day                     int64
Dep_Min                     int64
Journey_Day                 int64
Journey_Month               int64
Journey_Year                int64
Journey_DOW                 int64
dtype: object

In the next notebook EDA.ipynb, we will explore our data. For now we just save it.

In [100]:
data.to_csv("cleaned_data.csv")